## 阿里云SDK基本示例  
TODO:
- 请在这里补充阿里云的相关文档链接。
    mqtt_quick_start出处： https://help.aliyun.com/document_detail/261162.html
   其余连接在本文档中穿插，已经补充连接

- 写简单文档说明这里代码的功能。
  已经更新文档

### 阿里云基本MQTT Demo
使用设备页面的**MQTT参数**进行连接，注意不是DeviceSecret

In [11]:
import paho.mqtt.client as mqtt


MQTTHOST = "iot-06z009y9rnab11u.mqtt.iothub.aliyuncs.com"
MQTTPORT = 1883
mqttClient = mqtt.Client(client_id="i6faViFVrIm.lightTest|securemode=2,signmethod=hmacsha256,timestamp=1676854208595|")
mqttClient.username_pw_set(username="lightTest&i6faViFVrIm",
                           password="565b911af3521ac24715872677e594832cd6845d7ab8d1acbba54cef480d87e9")

print(mqttClient)

其设备Identifier是：
- device id，字符串i6faViFVrIm.light1，使用hmacsha256签名。timestamp什么用？
- username
- password

https://help.aliyun.com/document_detail/292635.html
这篇文章提到了如何计算MQTT签名，因为直接使用物联网页面获取设备信息时每次获取到的password不相同，**timestamp是hmacsha256的一个输入用于产生password**。

<br><br>
<font color=red>
    client id和username的作用？为什么需要两个？不能用client id做user name？
</font>
<br>

相关资料：
There  are three  required  fields  in  the CONNECT control  packet  for authentication:ClientId,Username, and Password. The ClientId is the unique identification of each client, which the broker uses to authorize the subscriber and publisher.And  the  pair  of Username and Password can  be  used  by the broker for both authentication and authorization.（Trampoline Over the Air: Breaking in IoT Devices Through MQTT Brokers）

The MQTT protocol requires the broker to disconnect the online  client  on  observing  a  new  client with  the  same ClientId. （Burglars’ IoT Paradise: Understanding and Mitigating Security Risks of General Messaging Protocols on IoT Clouds ）

为什么需要两个？

从上述资料来看clientid被broker用于授权subscriber和publisher，起码起到了将S与P分开的效果，<font color=red>但是深层次的设计目的没有了解到。</font>
我的理解是因为上面提到的MQTT协议的设计缺陷，ClientId可能需要进行一些处理变得比较复杂，而且有时username可能是公开信息，所以需要两个来提供更高的安全性。

不能用作client id做username？
我的理解为以下几点：
- 1.username方便记忆/输入
- 2.username可能是一个公开信息
- 3.clientId被用于区分subscriber和publisher


### 阿里云Linkkit
https://help.aliyun.com/document_detail/98291.html

### 在客户端订阅时需要注意的问题：
- 1.自定义Topic必须先在阿里云设备平台 '产品' 'Topic类列表' '自定义Topic' 点击定义Topic类。 https://help.aliyun.com/document_detail/73731.html
- 2.在直接使用mqtt而不是linkkit进行订阅时，自定义的Topic格式必须填写完整，比如 "/i6faViFVrIm/${deviceName}/user/test1" 而lk.to_full_topic函数功能是将前面两个参数进行补齐，以方便编码时只用输入"/user/test1"
- 3.如果客户端发送消息失败则需要检查"自定义Topic"类中权限，将权限设置为"发布和订阅"

In [16]:
# 连接MQTT服务器
def on_mqtt_connect():
    mqttClient.connect(MQTTHOST, MQTTPORT, 60)
    mqttClient.loop_start()


# publish 消息
def on_publish(topic, payload, qos):
    mqttClient.publish(topic, payload, qos)


# 消息处理函数
def on_message_come(lient, userdata, msg):
    print(msg.topic + " " + ":" + str(msg.payload))


# subscribe 消息
def on_subscribe():
    # 订阅监听自定义Topic
    mqttClient.subscribe("/i6faViFVrIm/lightTest/user/test1", 1)
    mqttClient.on_message = on_message_come  # 消息到来处理函数

def main():
    print("start note book")
    on_mqtt_connect()
    # 自定义Topic消息上行
    on_publish("/i6faViFVrIm/lightTest/user/test1", "Hello Aliyun!This is MQTT client", 1)
    on_subscribe()

In [15]:
main()

start note book
/i6faViFVrIm/lightTest/user/test1 :b'Hello Python!'
